In [170]:
import cv2
import numpy as np
import os
import csv
import math
from numpy import linalg as LA

In [2]:
MIN_DIST = 35
RESULTS_DIR = "results/"

In [3]:
def get_files(dirpath, ext):
    files = [s for s in os.listdir(dirpath)
         if os.path.isfile(os.path.join(dirpath, s)) and os.path.splitext(s)[1] == ext]
    files.sort()
    return files

def createDir(dirpath, ext):
    if os.path.exists(dirpath):
        for file in get_files(dirpath, ext):
            os.remove(os.path.join(dirpath, file))
    else:
        os.mkdir(dirpath)

In [139]:
def largestTrianglePointsIdxs(points):
    area = 0
    pointsIdxs = [0,0,0]
    if len(points) < 3:
        return None
    
    for i in range(len(points)-2):
        x1,y1 = points[i]
        for j in range(i+1,len(points)-1):
            x2,y2 = points[j]
            for k in range(j+1,len(points)):
                x3,y3 = points[k]
                if abs(0.5*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2))) > area :
                    area = abs(0.5*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)))
                    pointsIdxs = [i,j,k]

    return pointsIdxs

def findNearesPoints(points, keypoints):
    nearest_points = []
    for point in points:
        minDist = MIN_DIST
        minKeypoint = None
        for keypoint in keypoints:
            if math.sqrt((point[0] - keypoint[0])**2 + (point[1] - keypoint[1])**2) < minDist:
                minDist = math.sqrt((point[0] - keypoint[0])**2 + (point[1] - keypoint[1])**2)
                minKeypoint = keypoint
                
        if minKeypoint is None:
            return None
        
        nearest_points.append(minKeypoint)
        
    return nearest_points

In [201]:
def alignImages(table_file, regions_pattern, regions_table, 
                original_image, offset, rotate_matrix, border_points):
    keypoints_pattern = regions_pattern
    keypoints_table = regions_table
    
    print(len(keypoints_pattern), len(keypoints_table))
    if len(keypoints_pattern) < 3 or len(keypoints_table) < 3:
        print("Not enough elements")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", original_image)
        return

    pts1 = []
    pts2 = []
    new_matches = []
    for pIndex, keypoint_pattern in enumerate(keypoints_pattern):
        minDist = MIN_DIST
        pairIndex = 0
        for tIndex, keypoint_table in enumerate(keypoints_table):
            if math.sqrt((keypoint_pattern[0] - keypoint_table[0])**2 + (keypoint_pattern[1] - keypoint_table[1])**2) < minDist:
                if keypoint_table not in pts2:
                    pairIndex = tIndex
                    minDist = math.sqrt((keypoint_pattern[0] - keypoint_table[0])**2 
                                      + (keypoint_pattern[1] - keypoint_table[1])**2)
        if minDist < MIN_DIST:
            keypoint_table = keypoints_table[pairIndex]
            dmatch = cv2.DMatch(pIndex, pairIndex, 100)
            cv2.circle(original_image, (int(keypoint_pattern[0]), int(keypoint_pattern[1]) + offset), 
                       radius=MIN_DIST, color=(255, 0, 0), thickness=2)
            new_matches.append(dmatch)
            pts1.append(keypoint_pattern)
            pts2.append(keypoint_table)
        
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
            
    for point in keypoints_pattern:
        cv2.circle(original_image, (int(point[0]), int(point[1]) + offset), radius=10, color=(255, 0, 0), thickness=-1)
        
    for point in keypoints_table:
        cv2.circle(original_image, (int(point[0]), int(point[1]) + offset), radius=10, color=(0, 0, 255), thickness=-1)
        
    for point in pts2.astype(int):
        cv2.circle(original_image, (int(point[0]), int(point[1]) + offset), radius=10, color=(0, 255, 0), thickness=-1)
    
    print("Matches:", len(new_matches))
    if len(new_matches) < 3:
        print("Lack of matches")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", original_image)
        return 

    pointsIndexes = largestTrianglePointsIdxs(pts1)
    pts1_3 = np.float32([pts1[pointsIndexes[0]], pts1[pointsIndexes[1]], pts1[pointsIndexes[2]]])
    pts2_3 = np.float32([pts2[pointsIndexes[0]], pts2[pointsIndexes[1]], pts2[pointsIndexes[2]]])
    matrixAff = cv2.getAffineTransform(pts1_3, pts2_3) 
        
    pts_for_affin =  np.array([pts1_3], np.float32)
    dst_affin = cv2.transform(pts_for_affin, matrixAff).astype(int)
            
    fields_borders = []
    
    if abs(len(keypoints_pattern) - len(keypoints_table)) < len(keypoints_pattern) * 0.15 \
        and len(new_matches) > len(keypoints_table) * 0.8:
        
        for point in dst_affin[0]:
            cv2.circle(original_image, (int(point[0]), int(point[1]) + offset), radius=10, color=(255, 0, 255), thickness=-1)
        
        cv2.line(original_image, (dst_affin[0][0][0], dst_affin[0][0][1] + offset), 
                 (dst_affin[0][1][0], dst_affin[0][1][1] + offset), 
                 (255, 0, 255), 3, cv2.LINE_AA)
        cv2.line(original_image, (dst_affin[0][1][0], dst_affin[0][1][1] + offset),
                 (dst_affin[0][2][0], dst_affin[0][2][1] + offset),
                 (255, 0, 255), 3, cv2.LINE_AA)
        cv2.line(original_image, (dst_affin[0][2][0], dst_affin[0][2][1] + offset),
                 (dst_affin[0][0][0], dst_affin[0][0][1] + offset),
                 (255, 0, 255), 3, cv2.LINE_AA)        
        
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", original_image)
        
        for b_dict in border_points:
            field_type = b_dict['field_type']
            pts_border = b_dict['coords'].copy()
            pts_border = np.array(pts_border, np.int)
            
            dst_keypoints = findNearesPoints(pts_border, pts2)
            if dst_keypoints is None:
                pts_border_inv = np.array([[pts_border[0][0], pts_border[1][1]],
                                           [pts_border[1][0], pts_border[0][1]]], np.float32)

                pts_border_inv = np.array(pts_border_inv, np.float32)

                dst_keypoints_inv = findNearesPoints(pts_border_inv, pts2)
                print(dst_keypoints)
                if dst_keypoints is None:
                    pts_border_np =  np.array([pts_border], np.float32)
                    dst_border = cv2.transform(pts_border_np, matrixAff).astype(int)[0]
                else:
                    dst_border = np.array(dst_keypoints_inv.copy(), np.int) 
            else:
                dst_border = np.array(dst_keypoints.copy(), np.int)    
            
            pts_border[:,1] += offset
            dst_border[:,1] += offset

            original_image = cv2.rectangle(original_image, (dst_border[0][0], dst_border[0][1]),
                                (dst_border[1][0], dst_border[1][1]), (0, 255, 0), 5)
            
            fields_borders.append({'field_type': field_type, 'coords' : [[13,400],[1330,606]]})

    #     inv_rotate_matrix = cv2.invertAffineTransform(rotate_matrix)
    #     dst_border = cv2.transform(dst_border, inv_rotate_matrix).astype(int)
    
    cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", original_image)
    return fields_borders

In [202]:
def rotateIMG(image):
#     image = cv2.imread(jpgPath)
    midImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    BLACK_LIMIT = 200
    
    midImage[midImage < BLACK_LIMIT] = 0
    midImage[midImage > BLACK_LIMIT] = 255

    thresh = cv2.threshold(midImage, 190, 255, cv2.THRESH_BINARY_INV)[1]

    midImage = np.copy(thresh)
    
#     cv2.imwrite(rotateJpgPath[:-4] + "_thresh.jpg", midImage, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    
    dstImage = cv2.Canny(midImage, 80, 200, 3)
    
#     cv2.imwrite(rotateJpgPath[:-4] + "_canny.jpg", dstImage, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    
    lineimage = image.copy()
 
    lines = cv2.HoughLines(dstImage, 1, np.pi/180, 375)
    
    approvedCount = 0
    
    verticalCount = 0
    verticalSum = 0
    
    horizontalCount = 0
    horizontalSum = 0
    
    
    if lines is None:
        print("No lines")
        return
    
    for i in range(len(lines)):
        for rho, theta in lines[i]:
            lineAngle = theta / np.pi * 180
            
            if (lineAngle > 180):
                print(lineAngle)
                
            if (lineAngle < 80 or (lineAngle > 100 and lineAngle < 170) \
                or (lineAngle > 190 and lineAngle < 260) or lineAngle > 280):
                continue
            else:
                horizontalCount += 1
                horizontalSum += lineAngle   
                
                            
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(round(x0 + 4000 * (-b)))
                y1 = int(round(y0 + 4000 * a))
                x2 = int(round(x0 - 4000 * (-b)))
                y2 = int(round(y0 - 4000 * a))
#                 cv2.line(lineimage, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)


    average = horizontalSum / horizontalCount - 90 if horizontalCount > 0 else 0
    
    rotate = lineimage
        
    h, w = rotate.shape[:2]
    RotateMatrix = cv2.getRotationMatrix2D((w/2.0, h/2.0), average, 1)
    rotateImg = cv2.warpAffine(rotate, RotateMatrix, (w, h), borderValue=(255, 255, 255))

#     cv2.imwrite(rotateJpgPath, rotate, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    return rotateImg, RotateMatrix

In [203]:
def line_intersection(l1, l2):
    line1 = ([l1[0],l1[1]],[l1[2],l1[3]])
    line2 = ([l2[0],l2[1]],[l2[2],l2[3]])
    
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return (int(x), int(y))

def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def isPontOnLine(point, line):
    return abs(distance([line[0],line[1]], point) + distance([line[2],line[3]], point) \
        - distance([line[0],line[1]], [line[2],line[3]])) < 3


def getKeyPointsRegions(rotate_img, table_file):
    midImage = cv2.cvtColor(rotate_img, cv2.COLOR_BGR2GRAY)
            
    BLACK_LIMIT = 200
    
    midImage[midImage < BLACK_LIMIT] = 0
    midImage[midImage > BLACK_LIMIT] = 255

    thresh = cv2.threshold(midImage, 190, 255, cv2.THRESH_BINARY_INV)[1]

    horizontal = np.copy(thresh)
    vertical = np.copy(thresh)
    
    scale = 10
        
    horizontalCols = horizontal.shape[1]
    horizontalSize = horizontalCols / scale
    horizontalSize = int(horizontalSize)
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontalSize, 1))
    horizontal = cv2.morphologyEx(horizontal, cv2.MORPH_OPEN, horizontalStructure)

    
    verticalRows = vertical.shape[0]
    verticalSize = verticalRows / scale
    verticalSize = int(verticalSize)
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalSize))
    vertical = cv2.morphologyEx(vertical, cv2.MORPH_OPEN, verticalStructure)
    
    
    SE = cv2.getStructuringElement(cv2.MORPH_RECT, (1,3))
    horizontal = cv2.dilate(horizontal, SE, iterations=3)

    SE = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
    vertical = cv2.dilate(vertical, SE, iterations=5)
    
    lines_mask = horizontal + vertical
    
    
    points_image = rotate_img.copy()
    
    linesP = cv2.HoughLinesP(lines_mask, 1, np.pi/180, 150, None, 50, 20)
    

    if linesP is not None:    
        horizontalLines = [line[0] for line in linesP if abs(line[0][2] - line[0][0]) > abs(line[0][3] - line[0][1])]
        verticalLines = [line[0] for line in linesP if abs(line[0][2] - line[0][0]) < abs(line[0][3] - line[0][1])]
        
        count = 0
        for i in range(0, len(horizontalLines)):
            l = horizontalLines[i]
            l[0] -= 10
            l[2] += 10
            horizontalLines[i] = l
#             lineimg = cv2.line(points_image, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 2, cv2.LINE_AA)
#             cv2.imwrite(RESULTS_DIR + table_file + str(count) + "_points.jpg", lineimg)
            count += 1
            
        for i in range(0, len(verticalLines)):
            l = verticalLines[i]
            l[1] += 10
            l[3] -= 10
            verticalLines[i] = l
#             lineimg = cv2.line(points_image, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 2, cv2.LINE_AA)
#             cv2.imwrite(RESULTS_DIR + table_file + str(count) + "_points.jpg", lineimg)
            count += 1
            
#         print(count)
        keypoints = []
        count = 0
        for horizontal_line in horizontalLines:
            for vertical_line in verticalLines:
                point = line_intersection(horizontal_line, vertical_line)
                if point is not None:
                    if isPontOnLine(point, vertical_line) and isPontOnLine(point, horizontal_line):
                        keypoints.append(point)
                    
        hasChanges = True
        while hasChanges:
            hasChanges = False
            
            for keypoint in keypoints:
                if hasChanges:
                    break
                neighbours = []
                for neighbour in keypoints:
                    if keypoint == neighbour:
                        continue
                        
                    if math.sqrt((keypoint[0]-neighbour[0])**2 + (keypoint[1]-neighbour[1])**2) < 30:
                        neighbours.append(neighbour)

                if len(neighbours) > 0:
                    new_keypoint = keypoint
                    for neighbour in neighbours:
                        new_keypoint = (new_keypoint[0] + neighbour[0], 
                                        new_keypoint[1] + neighbour[1])
                    
                    new_keypoint = (int(new_keypoint[0] / (len(neighbours) + 1)), 
                                    int(new_keypoint[1] / (len(neighbours) + 1)))

                    keypoints = [point for point in keypoints if point not in neighbours and point != keypoint]
                    keypoints.append(new_keypoint)
                    
                    hasChanges = True
                    break
                        
        for point in keypoints:
            cv2.circle(points_image, point, radius=10, color=(0, 255, 0), thickness=-1)
                    
#         cv2.imwrite(RESULTS_DIR + table_file + "_points.jpg", points_image)
        
        return keypoints

In [204]:
def cropToPattern(rotate_pattern, rotate_table):
    cropped_table = rotate_table.copy()
    offset = 0
    if cropped_table.shape[0] > rotate_pattern.shape[0]:
        offset = cropped_table.shape[0] - rotate_pattern.shape[0]
        cropped_table = cropped_table[-rotate_pattern.shape[0]:,:,:]
        
    if cropped_table.shape[1] > rotate_pattern.shape[1]:
        cropped_table = cropped_table[:,:rotate_pattern.shape[1],:]
        
    return cropped_table, offset

In [207]:
createDir(RESULTS_DIR, '.jpg')
  
pattern_names = ["pattern_table.jpg"]
    
for table_file in get_files("tables/", ".jpg"):

    if table_file != "Screenshot_1.jpg":
        continue
    
    print(table_file)
    
    table_path = "tables/" + table_file
    rotateJpgPath = RESULTS_DIR + table_file[:-4] + "_r.jpg"

    table_im = cv2.imread(table_path, cv2.IMREAD_COLOR)
    rotated_table_im, rotate_matrix = rotateIMG(table_im.copy())
    
    for pattern_name in pattern_names:
        pattern_im = cv2.imread(pattern_name, cv2.IMREAD_COLOR)
        rotate_pattern, _ = rotateIMG(pattern_im)
        regions_pattern = getKeyPointsRegions(rotate_pattern, pattern_name)

        border_points = []
        with open(pattern_name[:-4] + '.csv') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                border_points.append({'field_type': row['field_type'], \
                                      'coords': [[row['x0'],row['y0']],[row['x1'],row['y1']]]})

        rotated_table_im, offset = cropToPattern(rotate_pattern, rotated_table_im)
        regions_table = getKeyPointsRegions(rotated_table_im, table_file)

        if regions_table is not None:
            borders = alignImages(table_path.split("/")[-1].split(".")[0], regions_pattern.copy(), 
                        regions_table.copy(), table_im.copy(), offset, rotate_matrix, border_points)
            if borders is not None:
                for border in borders:
                    print(border)
            else:
                print("Next pattern")


Screenshot_1.jpg
62 64
Matches: 62
None
{'field_type': 'Title', 'coords': [[13, 400], [1330, 606]]}
{'field_type': 'Number', 'coords': [[13, 400], [1330, 606]]}
{'field_type': 'Company', 'coords': [[13, 400], [1330, 606]]}
